# **Compound Meshes with Jagged Topology**

In [129]:
from polymesh import PolyData, TriMesh, Grid
from polymesh.space import StandardFrame
import numpy as np

In [130]:
A = StandardFrame(dim=3)
tri = TriMesh(size=(100, 100), shape=(10, 10), frame=A)
grid2d = Grid(size=(100, 100), shape=(10, 10), eshape='Q4', frame=A)
grid3d = Grid(size=(100, 100, 100), shape=(8, 6, 2), eshape='H8', frame=A)

In [131]:
np.vstack([tri.coords(), grid2d.coords(), grid3d.coords()]).shape

(410, 3)

In [132]:
np.vstack([tri.coords()]).shape

(100, 3)

In [133]:
np.vstack([grid2d.coords(), grid3d.coords()]).shape

(310, 3)

In [134]:
tri.topology().shape, grid2d.topology().shape, grid3d.topology().shape

((162, 3), (100, 4), (96, 8))

In [135]:
np.sum([tri.topology().shape[0], grid2d.topology().shape[0], grid3d.topology().shape[0]])

358

In [136]:
np.sum([grid2d.topology().shape[0], grid3d.topology().shape[0]])

196

In [137]:
mesh = PolyData(frame=A)

In [138]:
mesh['tri', 'T3'] = tri.move(np.array([0., 0., -200]))

In [139]:
mesh['grids', 'Q4'] = grid2d.move(np.array([0., 0., 200]))

In [140]:

mesh['grids', 'H8'] = grid3d

In [141]:
mesh['tri', 'T3'].pointdata['values'] = np.full(tri.coords().shape[0], 5.)
mesh['grids', 'Q4'].pointdata['values'] = np.full(grid2d.coords().shape[0], 10.)
mesh['grids', 'H8'].pointdata['values'] = np.full(grid3d.coords().shape[0], -5.)

In [142]:
mesh['tri', 'T3'].pointdata.values.to_numpy()

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [143]:
import pyvista as pv
from pyvista import themes

pv.set_plot_theme('paraview')

my_theme = themes.DarkTheme()
my_theme.color = 'red'
my_theme.lighting = False
my_theme.show_edges = True
my_theme.axes.box = True

pv.global_theme.load_theme(my_theme)

In [144]:
from pyvistaqt import BackgroundPlotter

plotter = BackgroundPlotter()
colors = ['red', 'green', 'blue']
for i, cb in enumerate(mesh.cellblocks()):
    data = cb.pointdata['values']
    plotter.add_mesh(cb.to_pv(), scalars=data)

In [145]:
plotter = BackgroundPlotter()
colors = ['red', 'green', 'blue']
for i, cb in enumerate(mesh.cellblocks()):
    m = cb.to_pv(scalars='values', fuse=False)[0]
    plotter.add_mesh(m)

In [146]:
mesh.plot()

In [147]:
mesh.frame

CartesianFrame
([[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]])

In [148]:
mesh['grids', 'H8'].frame

CartesianFrame
([[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]])

In [149]:
addr = mesh['grids', 'H8'].address
mesh[addr]

PolyData({'size': (100, 100, 100), 'shape': (8, 6, 2)})

In [150]:
mesh['grids'].topology()

TopologyArray
([[100, 111, 112, 101], [101, 112, 113, ... [384, 405, 408, 387, 385, 406, 409, 388]])

In [151]:
mesh['grids'].address

['grids']

In [152]:
mesh.pd

In [153]:
mesh['grids'].frame

CartesianFrame
([[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]])

In [154]:
mesh['tri']

PolyData({'T3': PolyData({'size': (100, 100), 'shape': (10, 10)})})

In [155]:
mesh['tri'].cd

In [156]:
mesh.to_standard_form()

PolyData({'tri': PolyData({'T3': PolyData({'size': (100, 100), 'shape': (10, 10)})}), 'grids': PolyData({'Q4': PolyData({'size': (100, 100), 'shape': (10, 10)}), 'H8': PolyData({'size': (100, 100, 100), 'shape': (8, 6, 2)})})})

In [157]:
triangles = mesh['tri'].detach()

In [158]:
grids = mesh['grids'].detach()

In [159]:
#grids.plot()

In [160]:
cb = list(grids.cellblocks(inclusive=True))

In [161]:
cb[0].topology().shape

(100, 4)

In [162]:
grids['H8'].parent.topology()

TopologyArray
([[100, 111, 112, 101], [101, 112, 113, ... [384, 405, 408, 387, 385, 406, 409, 388]])

In [163]:
grids.topology()

TopologyArray
([[100, 111, 112, 101], [101, 112, 113, ... [384, 405, 408, 387, 385, 406, 409, 388]])

In [164]:
for cb in grids.cellblocks():
    print(cb.address, cb.key)

['Q4'] Q4
['H8'] H8


In [165]:
for cb in grids['H8'].cellblocks(inclusive=True):
    print(cb.address, cb.key)

['H8'] H8


In [166]:
list(grids['H8'].cellblocks(inclusive=True))

[PolyData({})]

In [167]:
grids['H8'].__dict__

{'pointdata': None,
 'cell_index_manager': <polymesh.polydata.IndexManager at 0x2283d363760>,
 'celldata': <polymesh.cells.h8.H8 at 0x2283d363af0>,
 'celltype': polymesh.cells.h8.H8,
 '_frame': None,
 '_newaxis': 2,
 '_parent': PolyData({'Q4': PolyData({}), 'H8': PolyData({})}),
 '_config': LinkedDeepDict({'pv': LinkedDeepDict({'default': LinkedDeepDict({'show_edges': True})})}),
 'point_index_manager': <polymesh.polydata.IndexManager at 0x2283d3639a0>,
 '_root': PolyData({'Q4': PolyData({}), 'H8': PolyData({})}),
 '_locked': None,
 '_key': 'H8'}

In [168]:
mesh.coords(), mesh['grids', 'H8'].coords(from_cells=True), mesh['grids'].coords(from_cells=True)

(VectorBase([[   0.        ,    0.        , -200.        ],
             [  11.11111111,    0.        , -200.        ],
             [  22.22222222,    0.        , -200.        ],
             ...,
             [ 100.        ,  100.        ,    0.        ],
             [ 100.        ,  100.        ,   50.        ],
             [ 100.        ,  100.        ,  100.        ]]),
 VectorBase([[  0.        ,   0.        ,   0.        ],
             [  0.        ,   0.        ,  50.        ],
             [  0.        ,   0.        , 100.        ],
             [  0.        ,  16.66666667,   0.        ],
             [  0.        ,  16.66666667,  50.        ],
             [  0.        ,  16.66666667, 100.        ],
             [  0.        ,  33.33333333,   0.        ],
             [  0.        ,  33.33333333,  50.        ],
             [  0.        ,  33.33333333, 100.        ],
             [  0.        ,  50.        ,   0.        ],
             [  0.        ,  50.        ,  50.    

In [169]:
mesh.to_standard_form()

PolyData({'tri': PolyData({'T3': PolyData({'size': (100, 100), 'shape': (10, 10)})}), 'grids': PolyData({'Q4': PolyData({'size': (100, 100), 'shape': (10, 10)}), 'H8': PolyData({'size': (100, 100, 100), 'shape': (8, 6, 2)})})})

In [170]:
mesh.coords()

VectorBase([[   0.        ,    0.        , -200.        ],
            [  11.11111111,    0.        , -200.        ],
            [  22.22222222,    0.        , -200.        ],
            ...,
            [ 100.        ,  100.        ,    0.        ],
            [ 100.        ,  100.        ,   50.        ],
            [ 100.        ,  100.        ,  100.        ]])

In [171]:
mesh['grids', 'H8'].coords(from_cells=True)

VectorBase([[  0.        ,   0.        ,   0.        ],
            [  0.        ,   0.        ,  50.        ],
            [  0.        ,   0.        , 100.        ],
            [  0.        ,  16.66666667,   0.        ],
            [  0.        ,  16.66666667,  50.        ],
            [  0.        ,  16.66666667, 100.        ],
            [  0.        ,  33.33333333,   0.        ],
            [  0.        ,  33.33333333,  50.        ],
            [  0.        ,  33.33333333, 100.        ],
            [  0.        ,  50.        ,   0.        ],
            [  0.        ,  50.        ,  50.        ],
            [  0.        ,  50.        , 100.        ],
            [  0.        ,  66.66666667,   0.        ],
            [  0.        ,  66.66666667,  50.        ],
            [  0.        ,  66.66666667, 100.        ],
            [  0.        ,  83.33333333,   0.        ],
            [  0.        ,  83.33333333,  50.        ],
            [  0.        ,  83.33333333, 100.   

In [172]:
mesh['grids'].coords(from_cells=True)

VectorBase([[  0.        ,   0.        , 200.        ],
            [  0.        ,  10.        , 200.        ],
            [  0.        ,  20.        , 200.        ],
            [  0.        ,  30.        , 200.        ],
            [  0.        ,  40.        , 200.        ],
            [  0.        ,  50.        , 200.        ],
            [  0.        ,  60.        , 200.        ],
            [  0.        ,  70.        , 200.        ],
            [  0.        ,  80.        , 200.        ],
            [  0.        ,  90.        , 200.        ],
            [  0.        , 100.        , 200.        ],
            [ 10.        ,   0.        , 200.        ],
            [ 10.        ,  10.        , 200.        ],
            [ 10.        ,  20.        , 200.        ],
            [ 10.        ,  30.        , 200.        ],
            [ 10.        ,  40.        , 200.        ],
            [ 10.        ,  50.        , 200.        ],
            [ 10.        ,  60.        , 200.   

In [173]:
mesh.coords().shape

(410, 3)

In [174]:
mesh['grids'].is_root()

False

In [175]:
mesh['grids'].topology().shape[0]

196

In [176]:
mesh['grids', 'Q4'].source()

PolyData({'tri': PolyData({'T3': PolyData({'size': (100, 100), 'shape': (10, 10)})}), 'grids': PolyData({'Q4': PolyData({'size': (100, 100), 'shape': (10, 10)}), 'H8': PolyData({'size': (100, 100, 100), 'shape': (8, 6, 2)})})})

In [177]:
mesh.coords()

VectorBase([[   0.        ,    0.        , -200.        ],
            [  11.11111111,    0.        , -200.        ],
            [  22.22222222,    0.        , -200.        ],
            ...,
            [ 100.        ,  100.        ,    0.        ],
            [ 100.        ,  100.        ,   50.        ],
            [ 100.        ,  100.        ,  100.        ]])

In [178]:
mesh['grids', 'H8'].coords(from_cells=True)

VectorBase([[  0.        ,   0.        ,   0.        ],
            [  0.        ,   0.        ,  50.        ],
            [  0.        ,   0.        , 100.        ],
            [  0.        ,  16.66666667,   0.        ],
            [  0.        ,  16.66666667,  50.        ],
            [  0.        ,  16.66666667, 100.        ],
            [  0.        ,  33.33333333,   0.        ],
            [  0.        ,  33.33333333,  50.        ],
            [  0.        ,  33.33333333, 100.        ],
            [  0.        ,  50.        ,   0.        ],
            [  0.        ,  50.        ,  50.        ],
            [  0.        ,  50.        , 100.        ],
            [  0.        ,  66.66666667,   0.        ],
            [  0.        ,  66.66666667,  50.        ],
            [  0.        ,  66.66666667, 100.        ],
            [  0.        ,  83.33333333,   0.        ],
            [  0.        ,  83.33333333,  50.        ],
            [  0.        ,  83.33333333, 100.   

In [179]:
mesh['grids'].coords(from_cells=True).shape

(310, 3)

In [180]:
np.vstack([mesh['grids','Q4'].coords(from_cells=True), mesh['grids','H8'].coords(from_cells=True)]).shape

(310, 3)

In [181]:
mesh['tri'].coords(from_cells=True).shape

(100, 3)

In [182]:
mesh['tri'].coords(from_cells=True).shape, mesh['grids'].coords(from_cells=True).shape

((100, 3), (310, 3))

In [183]:
mesh.plot()